In [7]:
import pandas as pd

In [8]:
file_path = 'Data/airports.csv'

In [9]:
df = pd.read_csv(file_path)

In [10]:
df

,faa,name,lat,lon,alt,tz,dst,tzone
0,AAF,Apalachicola Regional Airport,29.727501,-85.027496,20,-5.0,A,America/New_York
1,AAP,Andrau Airpark,29.722500,-95.588303,79,-6.0,A,America/Chicago
2,ABE,Lehigh Valley International Airport,40.652100,-75.440804,393,-5.0,A,America/New_York
3,ABI,Abilene Regional Airport,32.411301,-99.681900,1791,-6.0,A,America/Chicago
4,ABL,Ambler Airport,67.106300,-157.856989,334,-9.0,A,America/Anchorage
...,...,...,...,...,...,...,...,...
1246,YKN,Chan Gurney Municipal Airport,42.916698,-97.385902,1306,-6.0,A,America/Chicago
1247,YNG,Youngstown Warren Regional Airport,41.260700,-80.679100,1192,-5.0,A,America/New_York
1248,YUM,Yuma MCAS/Yuma International Airport,32.656601,-114.606003,213,-7.0,N,America/Phoenix
1249,ZPH,Zephyrhills Municipal Airport,28.228201,-82.155899,90,-5.0,A,America/New_York
